In [1]:
import xarray as xr

In [5]:
ds = xr.open_dataset('/home/users/cturrell/documents/eddy_feedback/daily_datasets/jra55_djf_ep.nc')
ds.u.values

array([[[[-5.60547867e+01, -5.59922867e+01, -5.58047867e+01, ...,
          -5.56485367e+01, -5.58985367e+01, -5.60391617e+01],
         [-4.15860367e+01, -4.12735367e+01, -4.09766617e+01, ...,
          -4.17422867e+01, -4.18047867e+01, -4.17422867e+01],
         [-2.73204117e+01, -2.69454117e+01, -2.64766617e+01, ...,
          -2.79297867e+01, -2.78204117e+01, -2.76329117e+01],
         ...,
         [-6.13291168e+00, -6.21103668e+00, -6.25791168e+00, ...,
          -6.03916168e+00, -6.05478668e+00, -6.10166168e+00],
         [-2.53916168e+00, -2.57041168e+00, -2.58603668e+00, ...,
          -2.53916168e+00, -2.52353668e+00, -2.53916168e+00],
         [ 2.08583832e+00,  2.02333832e+00,  1.94521332e+00, ...,
           2.28896332e+00,  2.22646332e+00,  2.16396332e+00]],

        [[-3.91161308e+01, -3.93348808e+01, -3.94755058e+01, ...,
          -3.79755058e+01, -3.84130058e+01, -3.88036308e+01],
         [-2.75536308e+01, -2.76786308e+01, -2.77567558e+01, ...,
          -2.66317558e